**NOT&#258;**
C&#226;nd utiliz&#259;m [Binder](https://mybinder.org/), comanda ``python -m pip install --user plotly`` trebuie rulat&#259; &#238;n terminal &#238;nainte de pornirea **notebook**-ului!

# Atractorul Lorenz

&#206;n 1963, matematicianul &#351;i meteorologul [Edward R. Lorenz](https://en.wikipedia.org/wiki/Edward_Norton_Lorenz) introduce urm&#259;torul sistem de ecua&#355;ii diferen&#355;iale:

$$
\left\{
\begin{array}{l}
x^{\prime}=F(x,y,z)=\sigma\cdot(y-x),\\
y^{\prime}=G(x,y,z)=x\cdot(r-z)-y,\\
z^{\prime}=H(x,y,z)=x\cdot y-b\cdot z
\end{array}
\right.
\quad\mbox{unde }\sigma,\,r,\,b>0.
$$

Vezi lucrarea [Deterministic nonperiodic flow](https://journals.ametsoc.org/doi/abs/10.1175/1520-0469%281963%29020%3C0130%3ADNF%3E2.0.CO%3B2) din volumul [20, No. 2](https://journals.ametsoc.org/toc/atsc/20/2) al revistei [Journal of the Atmospheric Sciences](https://journals.ametsoc.org/toc/atsc/current). Ne vom referi la ea cu apelativul **Dnf**.

Avem nevoie de urm&#259;toarele [module Python 3](https://docs.python.org/3/reference/import.html):

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

In [ ]:
from decimal import Context, ROUND_HALF_EVEN, FloatOperation, setcontext, Decimal
contextul = Context(prec=128, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=1,
                    flags=[], traps=[FloatOperation])
setcontext(contextul)

Preg&#259;tim utilizarea bibliotecii grafice [Plotly](https://plot.ly/graphing-libraries/) &#238;n cadrul acestui [notebook](https://plot.ly/python/getting-started/#initialization-for-offline-plotting):

In [ ]:
init_notebook_mode(connected=True)

Conform **Dnf**, pagina 136, ne intereseaz&#259; urm&#259;toarele valori ale parametrilor: $\sigma=10,\,b=8/3$ (B. Saltzman) &#351;i $r=28$.

In [ ]:
def sist_lorenz(x, y, z, sigma=Decimal('10'), r=Decimal('28'), b=Decimal('8')/Decimal('3')):
    x_loc = Decimal(x)
    y_loc = Decimal(y)
    z_loc = Decimal(z)
    ec_x = sigma * (y_loc - x_loc)
    ec_y = x_loc * (r - z_loc) - y_loc
    ec_z = x_loc * y_loc - b * z_loc
    return ec_x,ec_y,ec_z

Folosim schema de aproximare numeric&#259; din **Dnf**, pagina 133. Astfel, **timpul** se estimeaz&#259; prin formula
$$
t_{k}=t_{0}+k\cdot\Delta t,\quad k\in\overline{0,n},\quad n\in\mathbb{N}.
$$

Presupun&#226;nd calculate m&#259;rimile
$$
x_{k}\sim x(t_{k}),\quad y_{k}\sim y(t_{k}),\quad z_{k}\sim z(t_{k}),
$$
determin&#259;m valorile auxiliare
$$
\left\{
\begin{array}{l}
x_{k}^{\scriptsize\mbox{aux}}=x_{k}+F(x_{k},y_{k},z_{k})\cdot\Delta t,\\
y_{k}^{\scriptsize\mbox{aux}}=y_{k}+G(x_{k},y_{k},z_{k})\cdot\Delta t,\\
z_{k}^{\scriptsize\mbox{aux}}=z_{k}+H(x_{k},y_{k},z_{k})\cdot\Delta t,\\
\end{array}
\right.
$$
respectiv
$$
\left\{
\begin{array}{l}
x_{k+1}^{\scriptsize\mbox{aux}}=x_{k}^{\scriptsize\mbox{aux}}+F(x_{k}^{\scriptsize\mbox{aux}},y_{k}^{\scriptsize\mbox{aux}},z_{k}^{\scriptsize\mbox{aux}})\cdot\Delta t,\\
y_{k+1}^{\scriptsize\mbox{aux}}=y_{k}^{\scriptsize\mbox{aux}}+G(x_{k}^{\scriptsize\mbox{aux}},y_{k}^{\scriptsize\mbox{aux}},z_{k}^{\scriptsize\mbox{aux}})\cdot\Delta t,\\
z_{k+1}^{\scriptsize\mbox{aux}}=z_{k}^{\scriptsize\mbox{aux}}+H(x_{k}^{\scriptsize\mbox{aux}},y_{k}^{\scriptsize\mbox{aux}},z_{k}^{\scriptsize\mbox{aux}})\cdot\Delta t.\\
\end{array}
\right.
$$

&#206;n sf&#226;r&#351;it,
$$
\left\{
\begin{array}{l}
x_{k+1}=\frac{1}{2}\cdot\left(x_{k}+x_{k+1}^{\scriptsize\mbox{aux}}\right),\\
y_{k+1}=\frac{1}{2}\cdot\left(y_{k}+y_{k+1}^{\scriptsize\mbox{aux}}\right),\\
z_{k+1}=\frac{1}{2}\cdot\left(z_{k}+z_{k+1}^{\scriptsize\mbox{aux}}\right).
\end{array}
\right.
$$

Conform discu&#355;iei din [Phil Austin, Lab. 6](https://clouds.eos.ubc.ca/~phil/numeric/html_files/lab6.html#5.-Numerical-Integration), metoda descris&#259; anterior este [metoda Heun](https://en.wikipedia.org/wiki/Heun%27s_method).

Pasul $h=\Delta t$ este:

In [ ]:
h = Decimal('0.01')

Num&#259;rul $n$ de itera&#355;ii este:

In [ ]:
n = 6_000

&#350;irurile de valori (&#238;ncepem cu datele ini&#355;iale):

In [ ]:
r = range(n + 1)
x = np.array(r, dtype=np.object_)
y = np.array(r, dtype=np.object_)
z = np.array(r, dtype=np.object_)
x_aux = np.array(r, dtype=np.object_)
y_aux = np.array(r, dtype=np.object_)
z_aux = np.array(r, dtype=np.object_)

Valorile ini&#355;iale sunt:

In [ ]:
x[0], y[0], z[0] = (Decimal('0'), Decimal('1'), Decimal('0'))

Calculul itera&#355;iilor (momentul ini&#355;ial este $t_{0}=0$ iar num&#259;rul de ordine al itera&#355;iei este $N$, vezi **Dnf**, paginile 133, 137):

In [ ]:
coef = Decimal('1') / Decimal('2')
for i in range(n):
    iter_x,iter_y,iter_z = sist_lorenz(x[i], y[i], z[i])
    x_aux[i] = x[i] + iter_x * h
    y_aux[i] = y[i] + iter_y * h
    z_aux[i] = z[i] + iter_z * h
    iter_x_aux,iter_y_aux,iter_z_aux = sist_lorenz(x_aux[i], y_aux[i], z_aux[i]) # captam si valorile intermediare
    x_aux[i + 1] = x_aux[i] + iter_x_aux * h
    y_aux[i + 1] = y_aux[i] + iter_y_aux * h
    z_aux[i + 1] = z_aux[i] + iter_z_aux * h
    x[i + 1] = coef * (x[i] + x_aux[i + 1])
    y[i + 1] = coef * (y[i] + y_aux[i + 1])
    z[i + 1] = coef * (z[i] + z_aux[i + 1])
    
    print("+++\nN = {0:d}, "
          "x({1}) = {2:.16f}, y({3}) = {4:.16f}, z({5}) = {6:.16f}".format(i, i, x[i], i, y[i], i, z[i]))

La itera&#355;ia num&#259;rul $J$ (pentru semnifica&#355;ia itera&#355;iilor **85**, **150**, vezi **Dnf**, pagina 137), unde

In [ ]:
J = 146

au fost ob&#355;inute valorile:

In [ ]:
print("x({0:d}) = {1:.16f}, y({2:d}) = {3:.16f}, z({4:d}) = {5:.16f}".format(J, x[J], J, y[J], J, z[J]))

Preg&#259;tirea vizualiz&#259;rii:

In [ ]:
traiectoria = go.Scatter3d(
# imaginea va putea fi salvata doar ca "png", vezi nota din "https://plot.ly/python/static-image-export/#vector-formats-svg-and-pdf"
              x=x, y=y, z=z, mode='markers', 
              marker=dict(color='rgb(127,0,255)', size=4, symbol='circle',
                          line=dict(color='rgb(255,245,0)', width=1),
                          opacity=0.8))
datele = [traiectoria]
aspectul = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

Vizualizarea interactiv&#259; a datelor:

In [ ]:
fig = go.Figure(data=datele, layout=aspectul)
iplot(fig)